# Ejercicio 4

Con los videos de youtube.com de cámara fija pedidos para esta clase, aplicar los algoritmos de detección de movimiento vistos en la teoría.

In [1]:
import cv2
cap = cv2.VideoCapture('../fotos/video.mp4')
backSub = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

max_height = 480

while True:
    ret, frame = cap.read()

    # Verifica si se ha leído correctamente un fotograma
    if not ret:
        break

    height, width = frame.shape[:2]
    ratio = max_height / height
    new_width = int(width * ratio)
    new_height = int(height * ratio)

    # Redimensiona el fotograma manteniendo la relación de aspecto
    frame_resized = cv2.resize(frame, (new_width, new_height))

    fgMask = backSub.apply(frame_resized)

    cv2.imshow('original', frame_resized)
    cv2.imshow('foregroundMask', fgMask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
# Función actualizada para realizar diferencia de fotogramas con normalización:
def process_frame_difference(new_image, prev_image, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 30, 255, cv2.THRESH_BINARY)

    # Convertir la imagen umbralizada a color para mantener la consistencia con el video original
    thresh_color = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

    return thresh_color

video_capture = cv2.VideoCapture('../fotos/video.mp4')

# Lee el primer fotograma
prev_frame = None
ret, prev_frame = video_capture.read()

# Itera sobre los fotogramas restantes
while True:
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # Procesa la diferencia entre fotogramas
    processed_frame = process_frame_difference(frame, prev_frame)

    # Muestra el fotograma procesado
    cv2.imshow('Processed Frame', processed_frame)

    # Sale del bucle si se presiona la tecla 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Actualiza el fotograma anterior
    prev_frame = frame.copy()

# Libera los recursos y cierra las ventanas
video_capture.release()
cv2.destroyAllWindows()